### LINES

In [ ]:
# Import neccsessary luibraries and first data

import numpy as np
import matplotlib.pyplot as plt
import math

DPI = 150
RAD_1 = 5

In [ ]:
# Functions 
def approx_points(river_X, river_Y, approx):
    gcd_point_X = int(river_X[0] * approx)
    gcd_point_Y = int(river_Y[0] * approx)
    for point_X, point_Y in zip(river_X, river_Y):
        gcd_point_X = math.gcd(gcd_point_X, int(point_X * approx))
        gcd_point_Y = math.gcd(gcd_point_Y, int(point_Y * approx))
    XY_gcd = math.gcd(gcd_point_X, gcd_point_Y)
    approx_river_X = [int(p * approx) / XY_gcd for p in river_X]
    approx_river_Y = [int(p * approx) / XY_gcd for p in river_Y]
    return approx_river_X, approx_river_Y

### Окружности

$$ (x - a)^{2} + (y - b)^{2} = r^2 $$
$$ x^{2} + y^{2} - 2ax - 2by + (b^{2} + a^{2}) = r^{2} $$
$$ A = -2a; B = -2b; C = b^{2} + a^{2}; $$

In [ ]:
def get_circle(p1, p2, p3):
    temp = p2[0] ** 2 + p2[1] ** 2
    bc = (p1[0] * p1[0] + p1[1] * p1[1] - temp) / 2
    cd = (temp - p3[0] * p3[0] - p3[1] * p3[1]) / 2
    det = (p1[0] - p2[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p2[1])
    
    if abs(det) < 1.0e-6:
        return (None, np.inf)
    
    cx = (bc * (p2[1] - p3[1]) - cd * (p1[1] - p2[1])) / det
    cy = ((p1[0] - p2[0]) * cd - (p2[0] - p3[0]) * bc) / det
    
    radius = np.sqrt((cx - p1[0])**2 + (cy - p1[1])**2)
    return ((cx, cy), radius)

In [ ]:
import geojson
with open('./pahma.geojson') as f:
    parsed_geojson = geojson.load(f)
    
selected_feature = 2
features = parsed_geojson['features'][selected_feature]
print(f"Count of features: {len(parsed_geojson['features'])}")
print(f"Selected feature: {selected_feature}")
geometry = features['geometry']
coordinates = geometry['coordinates']
print(f"Count of points: {len(coordinates)}")

river_X = list(zip(*coordinates))[0]
river_Y = list(zip(*coordinates))[1] 

river_X, river_Y = approx_points(river_X, river_Y, 5000)

In [ ]:
f = plt.figure()
f.set_dpi(DPI)

plt.axis('equal')

plt.plot(river_X, river_Y, 'black', linewidth = 1)

center, radius = get_circle((river_X[0],river_Y[0]), (river_X[1],river_Y[1]), (river_X[2],river_Y[2]))
if center is not None:
    circle = plt.Circle(center, radius, color='r', fill=False)
    plt.gcf().gca().add_artist(circle)

#circle_1 = plt.Circle((196400, 288080), 1, color='r', fill=False)
#plt.gca().add_artist(circle_1)

plt.show()